## Consultas a la base de datos

In [2]:
!pip install pymysql

In [5]:
import pymysql

In [17]:
class DataBase:
    def __init__(self):
        self.connection = pymysql.connect(
            host = "localhost",
            user = "root",
            password = "",
            db = "teleton"
        )
        self.cursor = self.connection.cursor()
    
        print("Conexión exitosa")
    
    def insert_users(self, nomina, nombre, puesto, tags, certificado, estatus):
            sql = f"INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('{nomina}', '{nombre}', '{puesto}', '{tags}', '{certificado}', '{estatus}')"
            print(sql)
            try:
                self.cursor.execute(sql)
                print(f"{nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def insert_documentos(self, Hash, Tipo, Nombre, Descripcion, Tags, TIME, Estatus):
            sql = f"INSERT INTO documentos (`Hash`, `Tipo`, `Nombre`, `Descripcion`, `Tags`, `TIME`, `Estatus`) VALUES ('{Hash}', '{Tipo}', '{Nombre}', '{Descripcion}', '{Tags}', '{TIME}', '{Estatus}')"
            print(sql)
            try:
                self.cursor.execute(sql)
                print(f"{Nombre} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
                
    def insert_firma(self, Hash, Doc_signed, TIME, Nomina):
            #sql = f"INSERT INTO firmas (`Hash`, `Doc_signed`, `TIME`, `Nomina`) VALUES ('{Hash}', `{Doc_signed}`, '{TIME}', '{Nomina}')"
            sql = """ INSERT INTO firmas  (Hash, Doc_signed, TIME, Nomina) VALUES (%s,%s,%s,%s)"""
            print(sql)
            try:
                self.cursor.execute(sql, (Hash, Doc_signed, TIME, Nomina))
                print(f"{Hash} ha sido añadido a la base de datos.")
            except Exception as e:
                print(e)
                
    def select(self, tabla, what, where, value):
        if what == "*":
            sql = f"SELECT {what} FROM {tabla}"
        else:
            sql = f"SELECT {what} FROM {tabla} where {where} = '{value}'"
        print(sql)
        self.cursor.execute(sql)
        val = self.cursor.fetchall()
        return val
        
    def update(self, tabla, what, value, where_col, where_val):
        sql = f"UPDATE {tabla} set {what} = '{value}' where {where_col} = '{where_val}'"
        self.cursor.execute(sql)
        print("Actualización exitosa")
        
    def delete(self, tabla, where, value):
        sql = f"DELETE FROM {tabla} where {where} = '{value}'"
        self.cursor.execute(sql)
        print("Eliminación exitosa")

database = DataBase()

Conexión exitosa


In [9]:
database.insert_users(nomina = "A01752069",
                nombre = "Jairo",
                puesto = "Estudiante",
                tags = "IDM",
                certificado = "pdjsnem",
                estatus = "Activo")

INSERT INTO users (`Nomina`, `Nombre`, `Puesto`, `Tags`, `Certificado`, `Estatus`) VALUES ('A01752069', 'Jairo', 'Estudiante', 'IDM', 'pdjsnem', 'Activo')
Jairo ha sido añadido a la base de datos.


In [15]:
def convertToBinaryData(filename):
    # Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData

In [16]:
archivo = convertToBinaryData("nombreDoc_nomina_fecha20220418_103207.sign")
database.insert_firma(Hash = "A01752068",
                Doc_signed = archivo,
                TIME = "Estudiante",
                Nomina = "IDM")

 INSERT INTO firmas  (Hash, Doc_signed, TIME, Nomina) VALUES (%s,%s,%s,%s)
A01752068 ha sido añadido a la base de datos.


In [25]:
database.insert_documentos(Hash = "jxajdljdljs", 
                           Tipo = "Certificado", 
                           Nombre = "Ventas.pdf", 
                           Descripcion = "Documento que bla bla",
                           Tags = "Ventas", 
                           TIME = "Ahorita", 
                           Estatus = "Activo")

INSERT INTO documentos (`Hash`, `Tipo`, `Nombre`, `Descripcion`, `Tags`, `TIME`, `Estatus`) VALUES ('jxajdljdljs', 'Certificado', 'Ventas.pdf', 'Documento que bla bla', 'Ventas', 'Ahorita', 'Activo')
jxajdljdljs ha sido añadido a la base de datos.


In [21]:
database.select(tabla = "firmas", 
                what = "Hash, Doc_signed", 
                where = "ID", 
                value = "1")

SELECT Hash, Doc_signed FROM firmas where ID = '1'


(('kndlajdlajdlaj',
  b'-----BEGIN CERTIFICATE-----\r\nMIIFzjCCA7YCAQAwDQYJKoZIhvcNAQENBQAwgawxCzAJBgNVBAYTAk1YMRQwEgYD\r\nVQQIDAtOdWV2byBMZcOzbjESMBAGA1UEBwwJTW9udGVycmV5MSIwIAYDVQQKDBlU\r\nZWNub2zDs2dpY28gZGUgTW9udGVycmV5MR0wGwYDVQQLDBRvcmdhbml6YXRpb25V\r\nbml0TmFtZTEPMA0GA1UEAwwGdGVjLm14MR8wHQYJKoZIhvcNAQkBFhBBMDE3NTA0\r\nNDNAdGVjLm14MB4XDTIyMDQwNzE0NDMyOVoXDTIzMDQwNzE0NDMyOVowgawxCzAJ\r\nBgNVBAYTAk1YMRQwEgYDVQQIDAtOdWV2byBMZcOzbjESMBAGA1UEBwwJTW9udGVy\r\ncmV5MSIwIAYDVQQKDBlUZWNub2zDs2dpY28gZGUgTW9udGVycmV5MR0wGwYDVQQL\r\nDBRvcmdhbml6YXRpb25Vbml0TmFtZTEPMA0GA1UEAwwGdGVjLm14MR8wHQYJKoZI\r\nhvcNAQkBFhBBMDE3NTA0NDNAdGVjLm14MIICIjANBgkqhkiG9w0BAQEFAAOCAg8A\r\nMIICCgKCAgEApNkmUo2STdFQ8HCwpuNxd96LSm4Jxv2uzfBJp46XlbnjlwJ6qFKB\r\nnzOAgUGFQOqCocOqIjHPYE/VVBYwJB/AvMK+EZ54txF9lc8Wor42fzCz4ewOykQa\r\nvE4PsheB5MucZ+UqyfO0KxPCaLLW6VcMi8QkeoVHbyB+d3frt4k6XQ4Dmtw11+m2\r\nDXboBxfPE0JBKbLv4jwqXdu5TWTbVNpN0BphqgXtM+LaBLxEQsA+vQSb4k+nZg1s\r\ncJdrT/Hj6oiWAU1I4eDfolA4WSA6cXbcR9xQKSYe9rpaJmEYm6lgDby91XTQI

In [72]:
from io import BytesIO
file_like = BytesIO(b'this is a sample bytearray')
type(file_like)

_io.BytesIO

In [79]:
F = database.select(tabla = "firmas", 
                what = "Hash, Doc_signed", 
                where = "ID", 
                value = "1")
with open("Temp.sign", 'wb') as file:
        file.write(F[0][1])
f = open("Temp.sign", 'rb').read()

SELECT Hash, Doc_signed FROM firmas where ID = '1'


In [44]:
#with open("nombreDoc_nomina_fecha20220418_103207.sign", 'rb') as file:
f = open("nombreDoc_nomina_fecha20220418_103207.sign", 'rb')
sigfile = f.read()

In [80]:
f

b'-----BEGIN CERTIFICATE-----\r\nMIIFzjCCA7YCAQAwDQYJKoZIhvcNAQENBQAwgawxCzAJBgNVBAYTAk1YMRQwEgYD\r\nVQQIDAtOdWV2byBMZcOzbjESMBAGA1UEBwwJTW9udGVycmV5MSIwIAYDVQQKDBlU\r\nZWNub2zDs2dpY28gZGUgTW9udGVycmV5MR0wGwYDVQQLDBRvcmdhbml6YXRpb25V\r\nbml0TmFtZTEPMA0GA1UEAwwGdGVjLm14MR8wHQYJKoZIhvcNAQkBFhBBMDE3NTA0\r\nNDNAdGVjLm14MB4XDTIyMDQwNzE0NDMyOVoXDTIzMDQwNzE0NDMyOVowgawxCzAJ\r\nBgNVBAYTAk1YMRQwEgYDVQQIDAtOdWV2byBMZcOzbjESMBAGA1UEBwwJTW9udGVy\r\ncmV5MSIwIAYDVQQKDBlUZWNub2zDs2dpY28gZGUgTW9udGVycmV5MR0wGwYDVQQL\r\nDBRvcmdhbml6YXRpb25Vbml0TmFtZTEPMA0GA1UEAwwGdGVjLm14MR8wHQYJKoZI\r\nhvcNAQkBFhBBMDE3NTA0NDNAdGVjLm14MIICIjANBgkqhkiG9w0BAQEFAAOCAg8A\r\nMIICCgKCAgEApNkmUo2STdFQ8HCwpuNxd96LSm4Jxv2uzfBJp46XlbnjlwJ6qFKB\r\nnzOAgUGFQOqCocOqIjHPYE/VVBYwJB/AvMK+EZ54txF9lc8Wor42fzCz4ewOykQa\r\nvE4PsheB5MucZ+UqyfO0KxPCaLLW6VcMi8QkeoVHbyB+d3frt4k6XQ4Dmtw11+m2\r\nDXboBxfPE0JBKbLv4jwqXdu5TWTbVNpN0BphqgXtM+LaBLxEQsA+vQSb4k+nZg1s\r\ncJdrT/Hj6oiWAU1I4eDfolA4WSA6cXbcR9xQKSYe9rpaJmEYm6lgDby91XTQIvku\r\nuo7vz8yDas1Uqrt

In [55]:
database.update(tabla = "users", 
                what = "Nombre", 
                value="Juanito", 
                where_col = "Nomina", 
                where_val = "A01752068")

Actualización exitosa


In [13]:
database.delete(tabla = "users", 
                where = "Nomina", 
                value="A0175068")

Eliminación exitosa


## Generación del certificado digital y llave privada

In [8]:
from OpenSSL import crypto, SSL

def cert_gen(
    emailAddress="emailAddress",
    commonName="commonName",
    countryName="NT",
    localityName="localityName",
    stateOrProvinceName="stateOrProvinceName",
    organizationName="organizationName",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds=0,
    validityEndInSeconds=10*365*24*60*60,
    KEY_FILE = "private.key",
    CERT_FILE="selfsigned.crt"):
    #can look at generated file using openssl:
    #openssl x509 -inform pem -in selfsigned.crt -noout -text
    # create a key pair
    k = crypto.PKey()
    k.generate_key(crypto.TYPE_RSA, 4096)
    # create a self-signed cert
    cert = crypto.X509()
    cert.get_subject().C = countryName
    cert.get_subject().ST = stateOrProvinceName
    cert.get_subject().L = localityName
    cert.get_subject().O = organizationName
    cert.get_subject().OU = organizationUnitName
    cert.get_subject().CN = commonName
    cert.get_subject().emailAddress = emailAddress
    cert.set_serial_number(serialNumber)
    cert.gmtime_adj_notBefore(0)
    cert.gmtime_adj_notAfter(validityEndInSeconds)
    cert.set_issuer(cert.get_subject())
    cert.set_pubkey(k)
    cert.sign(k, 'sha256')
    with open(CERT_FILE, "wt") as f:
        f.write(crypto.dump_certificate(crypto.FILETYPE_PEM, cert).decode("utf-8"))
    with open(KEY_FILE, "wt") as f:
        f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, k).decode("utf-8"))

In [9]:
cert_gen(emailAddress="A01750443@tec.mx",
    commonName="tec.mx",
    countryName="MX",
    localityName="Monterrey",
    stateOrProvinceName="Nuevo León",
    organizationName="Tecnológico de Monterrey",
    organizationUnitName="organizationUnitName",
    serialNumber=0,
    validityStartInSeconds = 0,
    validityEndInSeconds = 1*365*24*60*60, #UN AÑO
    KEY_FILE = "privateJERS_2.key",
    CERT_FILE="CertificadoJERS_2.crt")

## Cargar el certificado

In [30]:
import OpenSSL.crypto 
cert = OpenSSL.crypto.load_certificate(
    OpenSSL.crypto.FILETYPE_PEM, 
    open('CertificadoJERS_2.crt').read()
)
cert.get_notBefore()

b'20220418135149Z'

### Obteniendo valores del certificado

In [31]:
print(cert.get_signature_algorithm()) #Con qué algoritmo fue firmado
print(cert.get_notAfter()) #fecha de expiración
print(cert.get_pubkey())
print(cert.get_subject()) # Datos sobre el certificado

b'sha256WithRSAEncryption'
b'20230418135149Z'
<X509Name object '/C=MX/ST=Nuevo Le\xC3\xB3n/L=Monterrey/O=Tecnol\xC3\xB3gico de Monterrey/OU=organizationUnitName/CN=tec.mx/emailAddress=A01750443@tec.mx'>


## Revisar si un certificado coincide con la llave privada

In [33]:
def check_associate_cert_with_private_key(cert, private_key):
    """
    :type cert: str
    :type private_key: str
    :rtype: bool
    """
    try:
        private_key_obj = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, open(private_key).read())
    except OpenSSL.crypto.Error:
        raise Exception('private key is not correct: %s' % private_key)

    try:
        cert_obj = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, open(cert).read())
    except OpenSSL.crypto.Error:
        raise Exception('certificate is not correct: %s' % cert)

    context = OpenSSL.SSL.Context(OpenSSL.SSL.TLSv1_METHOD)
    context.use_privatekey(private_key_obj)
    context.use_certificate(cert_obj)
    try:
        context.check_privatekey()
        return True
    except OpenSSL.SSL.Error:
        return False

In [35]:
check_associate_cert_with_private_key("CertificadoJERS.crt", "privateJERS.key")

True

## Hash de cualquier documento

In [36]:
import sys
import hashlib

# BUF_SIZE is totally arbitrary, change for your app!
BUF_SIZE = 65536  # lets read stuff in 64kb chunks!

md5 = hashlib.md5()
sha1 = hashlib.sha256()
#hashlib.sha512

with open("../LSMLatinxAI_1.pdf", 'rb') as f:
    while True:
        data = f.read(BUF_SIZE)
        if not data:
            break
        md5.update(data)
        sha1.update(data)

print("MD5: {0}".format(md5.hexdigest()))
print("SHA1: {0}".format(sha1.hexdigest()))

MD5: d7a4be0864d8b40a1fe18c6139b4807a
SHA1: 7f042c1c98d9beb8ad45d803cfdce6dc25882eced8820ed91b19194b7e19e55e


## Generando y verificando un certificado

In [82]:
import signVerify

signVerify.gen_signature("privateJERS.key", bytes(sha1.hexdigest(), 'utf-8'))

<_io.BufferedWriter name='nombreDoc_nomina_fecha20220419_102246.sign'>

In [86]:
signVerify.verify("certificadoJERS.crt", bytes(sha1.hexdigest(), 'utf-8'), 'nombreDoc_nomina_fecha20220419_102246.sign')

Verification successful


In [87]:
signVerify.verify("certificadoJERS_2.crt", bytes(sha1.hexdigest(), 'utf-8'), 'nombreDoc_nomina_fecha20220419_102246.sign')

Verification failed
